In [3]:
import os
import requests
import json
import pandas as pd
from pandas import json_normalize

In [7]:
nyc_geo_df = pd.read_csv('nyc_geo_cleaned.csv')
# creating lat/lon column for api pulls
nyc_geo_df['LatLon'] = nyc_geo_df['Latitude'].astype(str) + '%2C' + nyc_geo_df['Longitude'].astype(str)
# converting latitude & longitude to strings for api pull
nyc_geo_df['Latitude'] = nyc_geo_df['Latitude'].astype(str)
nyc_geo_df['Longitude'] = nyc_geo_df['Longitude'].astype(str)
nyc_geo_df.head()

,Borough,Neighbourhood,Latitude,Longitude,LatLon
0,Bronx,Wakefield,40.89470517661,-73.84720052054902,40.89470517661%2C-73.84720052054902
1,Bronx,Co-op City,40.87429419303012,-73.82993910812398,40.87429419303012%2C-73.82993910812398
2,Bronx,Eastchester,40.887555677350775,-73.82780644716412,40.887555677350775%2C-73.82780644716412
3,Bronx,Fieldston,40.89543742690383,-73.90564259591682,40.89543742690383%2C-73.90564259591682
4,Bronx,Riverdale,40.890834493891305,-73.9125854610857,40.890834493891305%2C-73.9125854610857


## Foursquare API Requests

In [8]:
# fsq api info
foursquare_secret = os.environ["FSQ_API"]

In [9]:
# restaurant cat = 13065
# function for accessing desired information from the FSQ API

def fsq_locations (lat, lon, cat):
    
    """
        Returns a dataframe containing parsed json on a given location
    
    Parameters:
        lat (str): The latitude of the search area
        lon (str): The latitude of the search area
        cat (str): The poi category to return information on
    
    Returns:
        necessary framework to run the api_loop function
    """
    # construct url
    url= 'https://api.foursquare.com/v3/places/search?ll=' + lat + ',' + lon +'&radius=1000&categories=' + cat + '&limit=25&fields=name,distance,price,geocodes,rating'
    
    # create dictionary for headers
    headers = {"Accept": "application/json",
              'Authorization' : foursquare_secret}    
        
    # perform get request
    response = requests.get(url, headers=headers)

    # process request into usable JSON file
    fsq_response = json.loads(response.text)
    fsq_data_resto.append(fsq_response)
    
    # converts normalized JSON into a dataframe
    df = pd.json_normalize(fsq_data_resto,['results'])
    df['req_cat'] = cat
    
    # convert dataframe to global variable that can be accessed outside of the function
    global fsq_df
    fsq_df = df
    
    return


In [10]:
fsq_data_resto = []
def fsq_api_loop(cat):
    # Loop for iterating fsq_location function throughout geographic dataframe
    """
    Iterates through the nyc_geo_df, and the fsq_locations function.
    
    Parameters:
        cat (str): desired POI category
    Returns:
        fsq_df: a dataframe containing results from the api request

    """
    num_rows = nyc_geo_df.shape[0]
    row_count = 0


    while row_count < num_rows:
        fsq_locations((nyc_geo_df.iloc[row_count,2]),(nyc_geo_df.iloc[row_count,3]),cat)
        row_count +=1

In [17]:
fsq_api_loop('13065')
# exporting as csv
fsq_df.to_csv('fsq_poi.csv', index=False)

In [18]:
fsq_df.head()

,distance,name,price,rating,geocodes.main.latitude,geocodes.main.longitude,geocodes.roof.latitude,geocodes.roof.longitude,req_cat
0,127,Lollipops Gelato,1.0,8.6,40.893585,-73.843692,40.893585,-73.843692,13065
1,797,Ripe Kitchen and Bar,2.0,8.4,40.898196,-73.838821,40.898196,-73.838821,13065
2,821,Ali's Roti Shop,1.0,8.1,40.893950,-73.856803,40.893950,-73.856803,13065
3,983,Jimbo's Hamburger Palace,1.0,8.0,40.891853,-73.858478,40.891853,-73.858478,13065
4,454,Cooler Runnings Jamaican Restaurant,2.0,6.4,40.898173,-73.850254,40.898173,-73.850254,13065


## Yelp API Requests

In [11]:
yelp_secret = os.environ["YELP_API"]

In [12]:
# restaurant category = restaurants
# function for accessing desired information from the yelp API

def yelp_locations (lat, lon, cat):
    
    """
        Returns a dataframe containing parsed json on a given location
    
    Parameters:
        lat (str): The latitude of the search area
        lon (str): The latitude of the search area
        cat (str): The poi category to return information on
    
    Returns:
        necessary framework to run the api_loop function
    """
    # construct url
    url= 'https://api.yelp.com/v3/businesses/search?latitude=' + lat + '&longitude=' + lon +'&radius=1000&limit=25&categories=' + cat
    
    # create dictionary for headers
    headers = {"Accept": "application/json",
              'Authorization' : 'Bearer '+ yelp_secret}    
        
    # perform get request
    response = requests.get(url, headers=headers)

    # process request into usable JSON file
    yelp_response = json.loads(response.text)
    yelp_data_resto.append(yelp_response)
    
    # converts normalized JSON into a dataframe
    df = pd.json_normalize(yelp_data_resto,['businesses'])
    df['req_cat'] = cat
    
    # convert dataframe to global variable that can be accessed outside of the function
    global yelp_df
    yelp_df = df
    #print(yelp_response)
    return


In [13]:
yelp_data_resto=[]
def yelp_api_loop(cat):
    # Loop for iterating fsq_location function throughout geographic dataframe
    """
    Iterates through the nyc_geo_df, and the yelp_locations function.
    
    Parameters:
        cat (str): desired POI category
    Returns:
        yelp_df: a dataframe containing results from the api request

    """
    num_rows = nyc_geo_df.shape[0]
    row_count = 0


    while row_count < num_rows:
        yelp_locations((nyc_geo_df.iloc[row_count,2]),(nyc_geo_df.iloc[row_count,3]),cat)
        row_count +=1

In [14]:
yelp_api_loop('restaurants')
# exporting as csv
yelp_df.to_csv('yelp_poi.csv', index=False)

In [16]:
yelp_df.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,...,coordinates.longitude,location.address1,location.address2,location.address3,location.city,location.zip_code,location.country,location.state,location.display_address,req_cat
0,fLJw_HGD01RTQ_WqlJf6Ew,ripe-kitchen-and-bar-mount-vernon,Ripe Kitchen & Bar,https://s3-media3.fl.yelpcdn.com/bphoto/NCpeGV...,False,https://www.yelp.com/biz/ripe-kitchen-and-bar-...,287,"[{'alias': 'caribbean', 'title': 'Caribbean'}]",4.0,[delivery],...,-73.838821,151 W Sandford Blvd,,,Mount Vernon,10550,US,NY,"[151 W Sandford Blvd, Mount Vernon, NY 10550]",restaurants
1,iJci5KN7H39D3JXHiNBDKw,h-i-m-ital-health-food-market-bronx,H.I.M Ital Health Food Market,https://s3-media4.fl.yelpcdn.com/bphoto/Tze_Nm...,False,https://www.yelp.com/biz/h-i-m-ital-health-foo...,47,"[{'alias': 'healthmarkets', 'title': 'Health M...",4.5,[delivery],...,-73.854665,4374B White Plains Rd,,,Bronx,10466,US,NY,"[4374B White Plains Rd, Bronx, NY 10466]",restaurants
2,IcEafi60F4pyEHaM4lR4Wg,alis-roti-shop-bronx,Ali's Roti Shop,https://s3-media2.fl.yelpcdn.com/bphoto/6SQMma...,False,https://www.yelp.com/biz/alis-roti-shop-bronx?...,104,"[{'alias': 'trinidadian', 'title': 'Trinidadia...",4.0,[],...,-73.856840,4220 White Plains Rd,,,Bronx,10466,US,NY,"[4220 White Plains Rd, Bronx, NY 10466]",restaurants
3,h5dm87qaqHeTBGcqTi9hWg,paulas-soul-cafe-bronx,Paula's Soul Cafe,https://s3-media4.fl.yelpcdn.com/bphoto/yfGmup...,False,https://www.yelp.com/biz/paulas-soul-cafe-bron...,202,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",3.0,"[delivery, pickup]",...,-73.855680,746 E 233rd St,,,Bronx,10466,US,NY,"[746 E 233rd St, Bronx, NY 10466]",restaurants
4,_XMhEsFSFUQsuKae7zzFZg,jerk-house-bronx,Jerk House,https://s3-media4.fl.yelpcdn.com/bphoto/Jx4v64...,False,https://www.yelp.com/biz/jerk-house-bronx?adju...,62,"[{'alias': 'caribbean', 'title': 'Caribbean'}]",3.5,"[delivery, pickup]",...,-73.856480,4246 White Plains,,None,Bronx,10466,US,NY,"[4246 White Plains, Bronx, NY 10466]",restaurants


## Google Places

In [ ]:
# google places api info
gp_secret = os.environ["gp_api"]

In [ ]:
# restaurant category in google places = restaurant
# function for accessing desired information from the Google Places API

def gp_locations (lat_lon, cat):
    
    """
        Returns a dataframe containing parsed json on a given location
    
    Parameters:
        lat_lon (str): The latitude & longitude of the search area (pre-formatted to fit google's preferred formatting)
        cat (str): The poi category to return information on
    
    Returns:
        necessary framework to run the api_loop function
    """
    # construct url
    url= 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=' + lat_lon + '&radius=1000&type='+ cat + '&key=' + gp_secret
    
    # create dictionary for headers
    headers = {}
    #payload = {}
        
    # perform get request
    response = requests.get(url, headers=headers)

    # process request into usable JSON file
    gp_response = json.loads(response.text)
    gp_data_resto.append(gp_response)
    
    # converts normalized JSON into a dataframe
    df = pd.json_normalize(gp_data_resto,['results'])
    df['req_cat'] = cat
    
    # convert dataframe to global variable that can be accessed outside of the function
    global gp_df
    gp_df = df
    
    return


In [ ]:
gp_data_resto=[]
def gp_api_loop(cat):
    # Loop for iterating fsq_location function throughout geographic dataframe
    """
    Iterates through the nyc_geo_df, and the gp_locations function.
    
    Parameters:
        cat (str): desired POI category
    Returns:
        gp_df: a dataframe containing results from the api request

    """
    num_rows = nyc_geo_df.shape[0]
    row_count = 0


    while row_count < num_rows:
        gp_locations((nyc_geo_df.iloc[row_count,4]),cat)
        row_count +=1

In [ ]:
gp_api_loop('restaurant')

In [ ]:
gp_df

,business_status,icon,icon_background_color,icon_mask_base_uri,name,photos,place_id,price_level,rating,reference,...,geometry.location.lng,geometry.viewport.northeast.lat,geometry.viewport.northeast.lng,geometry.viewport.southwest.lat,geometry.viewport.southwest.lng,opening_hours.open_now,plus_code.compound_code,plus_code.global_code,permanently_closed,req_cat
0,OPERATIONAL,https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,Ripe Kitchen & Bar,"[{'height': 2471, 'html_attributions': ['<a hr...",ChIJ9aOdYiyNwokRS49czrNTQo4,2.0,4.3,ChIJ9aOdYiyNwokRS49czrNTQo4,...,-73.838855,40.899498,-73.837452,40.896800,-73.840150,True,"V5X6+7F Mount Vernon, NY, USA",87G8V5X6+7F,NaN,restaurant
1,OPERATIONAL,https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,New China Garden,"[{'height': 4032, 'html_attributions': ['<a hr...",ChIJcVDAYNfywokRPUaKrm0UBpg,1.0,3.9,ChIJcVDAYNfywokRPUaKrm0UBpg,...,-73.853364,40.899306,-73.852100,40.896608,-73.854798,True,"V4XW+5M The Bronx, NY, USA",87G8V4XW+5M,NaN,restaurant
2,OPERATIONAL,https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,Dunkin',"[{'height': 3000, 'html_attributions': ['<a hr...",ChIJ--58aynzwokRqSvaRDKwX1M,1.0,3.8,ChIJ--58aynzwokRqSvaRDKwX1M,...,-73.849089,40.891936,-73.847664,40.889239,-73.850362,True,"V5R2+59 New York, NY, USA",87G8V5R2+59,NaN,restaurant
3,OPERATIONAL,https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,Subway,"[{'height': 766, 'html_attributions': ['<a hre...",ChIJAy-ibCnzwokRdmkKGd1Yzhw,1.0,3.6,ChIJAy-ibCnzwokRdmkKGd1Yzhw,...,-73.849152,40.891828,-73.847659,40.889130,-73.850357,True,"V5R2+58 New York, NY, USA",87G8V5R2+58,NaN,restaurant
4,OPERATIONAL,https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,Popeyes Louisiana Kitchen,"[{'height': 1192, 'html_attributions': ['<a hr...",ChIJXVacFSvzwokRa3VpQDuwRDE,1.0,3.8,ChIJXVacFSvzwokRa3VpQDuwRDE,...,-73.843383,40.890762,-73.842012,40.888064,-73.844710,True,"V5Q4+QJ New York, NY, USA",87G8V5Q4+QJ,NaN,restaurant
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5614,OPERATIONAL,https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,Romeo's Pizza,"[{'height': 4032, 'html_attributions': ['<a hr...",ChIJTSbjhExOwokRYksSKMsMcu8,1.0,4.3,ChIJTSbjhExOwokRYksSKMsMcu8,...,-74.079342,40.626117,-74.078021,40.623419,-74.080719,True,"JWFC+W7 New York, NY, USA",87G7JWFC+W7,NaN,restaurant
5615,OPERATIONAL,https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,Amins fried chiken,"[{'height': 3024, 'html_attributions': ['<a hr...",ChIJfU46JlhPwokR0VHLSCHO4bg,NaN,4.8,ChIJfU46JlhPwokR0VHLSCHO4bg,...,-74.088093,40.613042,-74.086844,40.610344,-74.089542,True,"JW66+MQ New York, NY, USA",87G7JW66+MQ,NaN,restaurant
5616,OPERATIONAL,https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,Taco Bell,"[{'height': 2252, 'html_attributions': ['<a hr...",ChIJrZlG6k9PwokR--BvOpa_p4I,1.0,3.9,ChIJrZlG6k9PwokR--BvOpa_p4I,...,-74.072007,40.617294,-74.070708,40.614596,-74.073406,True,"JW8H+95 New York, NY, USA",87G7JW8H+95,NaN,restaurant
5617,OPERATIONAL,https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,Palm Grill,NaN,ChIJo--U6TRPwokR6SMBlBfPm6E,NaN,3.5,ChIJo--U6TRPwokR6SMBlBfPm6E,...,-74.072582,40.615333,-74.071251,40.612635,-74.073949,NaN,"JW7G+HX Staten Island, NY, USA",87G7JW7G+HX,NaN,restaurant


In [ ]:
# exporting as csv
gp_df.to_csv('google_places_poi.csv', index=False)